# Lab | Data Cleaning and Formatting

In this lab, we will be working with the customer data from an insurance company, which can be found in the CSV file located at the following link: https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file1.csv


# Challenge 1: Data Cleaning and Formatting

## Exercise 1: Cleaning Column Names

To ensure consistency and ease of use, standardize the column names of the dataframe. Start by taking a first look at the dataframe and identifying any column names that need to be modified. Use appropriate naming conventions and make sure that column names are descriptive and informative.

*Hint*:
- *Column names should be in lower case*
- *White spaces in column names should be replaced by `_`*
- *`st` could be replaced for `state`*

In [1]:
# Your code here
import pandas as pd

# Load your dataset (replace with your actual file)
df = pd.read_csv('file1.csv')

# Create a dictionary to map old column names to new standardized names
column_name_mapping = {
    # General naming conventions
    'ST': 'state',
    'Customer Lifetime Value': 'customer_lifetime_value',
    'Income': 'income',
    'Monthly Premium Auto': 'monthly_premium_auto',
    'Number of Open Complaints': 'number_of_open_complaints',
    'Policy Type': 'policy_type',
    'Vehicle Class': 'vehicle_class',
    'Total Claim Amount': 'total_claim_amount',
    
    # Add more mappings for your specific columns
    # Format: 'OldColumnName': 'new_column_name'
}

# Apply the renaming
df.rename(columns=column_name_mapping, inplace=True)

# For any remaining columns not in our mapping, apply general cleaning rules
def clean_column_name(col):
    col = col.lower()  # Convert to lowercase
    col = col.replace(' ', '_')  # Replace spaces with underscores
    col = col.replace('-', '_')  # Replace hyphens with underscores
    col = col.replace('(', '')  # Remove special characters
    col = col.replace(')', '')
    col = col.replace('/', '_')
    return col

df.columns = [clean_column_name(col) for col in df.columns]

# Verify the changes
print("Updated column names:")
print(df.columns.tolist())

# Check for any remaining problematic columns
print("\nColumns that might still need attention:")
for col in df.columns:
    if any(x in col for x in [' ', '-', '(', ')', '/']) or col != col.lower():
        print(f"- {col} (needs further cleaning)")

Updated column names:
['customer', 'state', 'gender', 'education', 'customer_lifetime_value', 'income', 'monthly_premium_auto', 'number_of_open_complaints', 'policy_type', 'vehicle_class', 'total_claim_amount']

Columns that might still need attention:


## Exercise 2: Cleaning invalid Values

The dataset contains columns with inconsistent and incorrect values that could affect the accuracy of our analysis. Therefore, we need to clean these columns to ensure that they only contain valid data.

Note that this exercise will focus only on cleaning inconsistent values and will not involve handling null values (NaN or None).

*Hint*:
- *Gender column contains various inconsistent values such as "F", "M", "Femal", "Male", "female", which need to be standardized, for example, to "M" and "F".*
- *State abbreviations be can replaced with its full name, for example "AZ": "Arizona", "Cali": "California", "WA": "Washington"*
- *In education, "Bachelors" could be replaced by "Bachelor"*
- *In Customer Lifetime Value, delete the `%` character*
- *In vehicle class, "Sports Car", "Luxury SUV" and "Luxury Car" could be replaced by "Luxury"*

In [11]:
# Your code here
import pandas as pd
from us import states  # For state abbreviation handling (install with 'pip install us')

# Load your dataset
df = pd.read_csv('file1.csv')

# 1. Clean Gender Column
df['GENDER'] = df['GENDER'].str.lower().str[0]  # Take first character and lowercase
df['GENDER'] = df['GENDER'].replace({'u': None, 'n': None})  # Handle unknown/neutral
df['GENDER'] = df['GENDER'].map({'m': 'm', 'f': 'f'})  # Standardize to m/f

# 2. Convert State Abbreviations to Full Names
def get_state_full_name(abbr):
    try:
        return states.lookup(abbr).name if pd.notna(abbr) else None
    except:
        return abbr  # Return original if conversion fails

df['ST'] = df['ST'].str.upper().apply(get_state_full_name)

# 3. Standardize Education Column
df['Education'] = df['Education'].str.replace('Bachelors', 'Bachelor')
df['Education'] = df['Education'].str.title()  # Standardize capitalization

# 4. Clean Customer Lifetime Value (remove % and convert to numeric)
df['Customer Lifetime Value'] = (
    df['Customer Lifetime Value']
    .astype(str)
    .str.replace('%', '')
    .astype(float)
)

# 5. Standardize Vehicle Class
luxury_vehicles = ['sports car', 'luxury suv', 'luxury car']
df['Vehicle Class'] = (
    df['Vehicle Class']
    .str.lower()
    .replace(luxury_vehicles, 'Luxury')
    .str.title()  # Proper case for other categories
)

# Verification
print("Cleaning results:")
print(f"Gender values: {df['GENDER'].unique()}")
print(f"State sample: {df['ST'].head(3)}")
print(f"Education values: {df['Education'].unique()}")
print(f"CLV sample: {df['Customer Lifetime Value'].head(3)}")
print(f"Vehicle classes: {df['Vehicle Class'].value_counts()}")

Cleaning results:
Gender values: [nan 'f' 'm']
State sample: 0    Washington
1       Arizona
2        Nevada
Name: ST, dtype: object
Education values: ['Master' 'Bachelor' 'High School Or Below' 'College' 'Doctor' nan]
CLV sample: 0           NaN
1     697953.59
2    1288743.17
Name: Customer Lifetime Value, dtype: float64
Vehicle classes: Vehicle Class
Four-Door Car    576
Two-Door Car     205
Suv              199
Luxury            91
Name: count, dtype: int64


## Exercise 3: Formatting data types

The data types of many columns in the dataset appear to be incorrect. This could impact the accuracy of our analysis. To ensure accurate analysis, we need to correct the data types of these columns. Please update the data types of the columns as appropriate.

It is important to note that this exercise does not involve handling null values (NaN or None).

*Hint*:
- *Customer lifetime value should be numeric*
- *Number of open complaints has an incorrect format. Look at the different values it takes with `unique()` and take the middle value. As an example, 1/5/00 should be 5. Number of open complaints is a string - remember you can use `split()` to deal with it and take the number you need. Finally, since it should be numeric, cast the column to be in its proper type.*

In [13]:
# Your code here
# Remove % signs and any other non-numeric characters, then convert to float
df['Customer Lifetime Value'] = (
    df['Customer Lifetime Value']
    .astype(str)
    .str.replace('[%,$]', '', regex=True)  # Remove % and $ symbols
    .str.strip()  # Remove whitespace
    .replace('', '0')  # Handle empty strings
    .astype(float)
)

# Extract the middle number from format X/Y/Z
df['Number of Open Complaints'] = (
    df['Number of Open Complaints']
    .astype(str)
    .str.split('/')  # Split by forward slash
    .apply(lambda x: x[1] if len(x) == 3 else None)  # Take middle value
    .astype(float)  # Convert to numeric
    .fillna(0)  # Replace missing with 0 if needed
    .astype(int)  # Convert to integer
)

import pandas as pd

# Clean Customer Lifetime Value
df['Customer Lifetime Value'] = (
    df['Customer Lifetime Value']
    .astype(str)
    .str.replace('[^0-9.]', '', regex=True)  # Keep only numbers and decimal point
    .replace('', '0')
    .astype(float)
)

# Process Open Complaints
def extract_complaint_value(complaint_str):
    if pd.isna(complaint_str):
        return 0
    parts = str(complaint_str).split('/')
    return int(parts[1]) if len(parts) == 3 else 0

df['Number of Open Complaints'] = (
    df['Number of Open Complaints']
    .apply(extract_complaint_value)
)

# Verification
print("Data Types After Cleaning:")
print(df[['Customer Lifetime Value', 'Number of Open Complaints']].dtypes)

print("\nSample Values:")
print(df[['Customer Lifetime Value', 'Number of Open Complaints']].head())

print("\nValue Counts for Open Complaints:")
print(df['Number of Open Complaints'].value_counts().sort_index())



Data Types After Cleaning:
Customer Lifetime Value      float64
Number of Open Complaints      int64
dtype: object

Sample Values:
   Customer Lifetime Value  Number of Open Complaints
0                     0.00                          0
1                697953.59                          0
2               1288743.17                          0
3                764586.18                          0
4                536307.65                          0

Value Counts for Open Complaints:
Number of Open Complaints
0    4008
Name: count, dtype: int64


## Exercise 4: Dealing with Null values

Identify any columns with null or missing values. Identify how many null values each column has. You can use the `isnull()` function in pandas to find columns with null values.

Decide on a strategy for handling the null values. There are several options, including:

- Drop the rows or columns with null values
- Fill the null values with a specific value (such as the column mean or median for numerical variables, and mode for categorical variables)
- Fill the null values with the previous or next value in the column
- Fill the null values based on a more complex algorithm or model (note: we haven't covered this yet)

Implement your chosen strategy to handle the null values. You can use the `fillna()` function in pandas to fill null values or `dropna()` function to drop null values.

Verify that your strategy has successfully handled the null values. You can use the `isnull()` function again to check if there are still null values in the dataset.

Remember to document your process and explain your reasoning for choosing a particular strategy for handling null values.

After formatting data types, as a last step, convert all the numeric variables to integers.

In [15]:
# Your code here
import pandas as pd

# Load your cleaned dataset
df = pd.read_csv('file1.csv')

# Strategy: Drop columns with >30% null values, rows with any remaining nulls
print("Initial null values:")
print(df.isnull().sum())

# Drop columns with excessive nulls (threshold: 30%)
null_percent = df.isnull().mean() * 100
cols_to_drop = null_percent[null_percent > 30].index
df = df.drop(columns=cols_to_drop)

print(f"\nDropped columns with >30% nulls: {list(cols_to_drop)}")

# Drop rows with any remaining null values
initial_rows = len(df)
df = df.dropna()
final_rows = len(df)

print(f"\nDropped {initial_rows - final_rows} rows with null values")
print("\nRemaining null values:")
print(df.isnull().sum())

# Verify no nulls remain
assert df.isnull().sum().sum() == 0, "Null values still exist!"

### 2. Convert Numeric Variables to Integers
# First identify numeric columns (excluding categorical codes)
numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns

# Convert to integers where possible (losing decimals)
for col in numeric_cols:
    try:
        df[col] = df[col].astype(int)
        print(f"Converted {col} to integer")
    except ValueError:
        print(f"Could not convert {col} to integer (contains decimals)")
        # Optional: Round decimals first if desired
        # df[col] = df[col].round().astype(int)

### 3. Final Verification
print("\nFinal data types:")
print(df.dtypes)

print("\nSample data:")
print(df.head())

Initial null values:
Customer                     2937
ST                           2937
GENDER                       3054
Education                    2937
Customer Lifetime Value      2940
Income                       2937
Monthly Premium Auto         2937
Number of Open Complaints    2937
Policy Type                  2937
Vehicle Class                2937
Total Claim Amount           2937
dtype: int64

Dropped columns with >30% nulls: ['Customer', 'ST', 'GENDER', 'Education', 'Customer Lifetime Value', 'Income', 'Monthly Premium Auto', 'Number of Open Complaints', 'Policy Type', 'Vehicle Class', 'Total Claim Amount']

Dropped 0 rows with null values

Remaining null values:
Series([], dtype: float64)

Final data types:
Series([], dtype: object)

Sample data:
Empty DataFrame
Columns: []
Index: [0, 1, 2, 3, 4]


## Exercise 5: Dealing with duplicates

Use the `.duplicated()` method to identify any duplicate rows in the dataframe.

Decide on a strategy for handling the duplicates. Options include:
- Dropping all duplicate rows
- Keeping only the first occurrence of each duplicated row
- Keeping only the last occurrence of each duplicated row
- Dropping duplicates based on a subset of columns
- Dropping duplicates based on a specific column

Implement your chosen strategy using the `drop_duplicates()` function.

Verify that your strategy has successfully handled the duplicates by checking for duplicates again using `.duplicated()`.

Remember to document your process and explain your reasoning for choosing a particular strategy for handling duplicates.

Save the cleaned dataset to a new CSV file.

*Hint*: *after dropping duplicates, reset the index to ensure consistency*.

In [17]:
# Your code here
import pandas as pd

# Load your cleaned dataset
df = pd.read_csv('file1.csv')

# Before cleaning report
initial_count = len(df)
duplicate_count = df.duplicated().sum()
print(f"Initial rows: {initial_count}")
print(f"Duplicate rows found: {duplicate_count}")

# Strategic duplicate removal - keep first occurrence
df_clean = df.drop_duplicates(keep='first')

# Alternative options:
# df_clean = df.drop_duplicates(keep='last')  # Keep last occurrence
# df_clean = df.drop_duplicates(keep=False)  # Remove ALL duplicates

# Verification
remaining_duplicates = df_clean.duplicated().sum()
rows_removed = initial_count - len(df_clean)

print(f"\nAfter cleaning:")
print(f"Rows removed: {rows_removed}")
print(f"Remaining rows: {len(df_clean)}")
print(f"Remaining duplicates: {remaining_duplicates}")

# Final check!
assert remaining_duplicates == 0, "Duplicate removal failed!"

### 2. Save newly Cleaned Dataset
output_file = 'cleaned_data_no_duplicates.csv'
df_clean.to_csv(output_file, index=False)
print(f"\nSaved cleaned data to: {output_file}")

### 3. Advanced Verification (Optional)
# Check for duplicates on specific columns only
# key_columns = ['customer_id', 'policy_number']
# if df_clean.duplicated(subset=key_columns).sum() > 0:
#     print("Warning: Potential business logic duplicates exist")

Initial rows: 4008
Duplicate rows found: 2936

After cleaning:
Rows removed: 2936
Remaining rows: 1072
Remaining duplicates: 0

Saved cleaned data to: cleaned_data_no_duplicates.csv


# Bonus: Challenge 2: creating functions on a separate `py` file

Put all the data cleaning and formatting steps into functions, and create a main function that performs all the cleaning and formatting.

Write these functions in separate .py file(s). By putting these steps into functions, we can make the code more modular and easier to maintain.

*Hint: autoreload module is a utility module in Python that allows you to automatically reload modules in the current session when changes are made to the source code. This can be useful in situations where you are actively developing code and want to see the effects of changes you make without having to constantly restart the Python interpreter or Jupyter Notebook kernel.*

In [ ]:
# Your code here

# Bonus: Challenge 3: Analyzing Clean and Formated Data

You have been tasked with analyzing the data to identify potential areas for improving customer retention and profitability. Your goal is to identify customers with a high policy claim amount and a low customer lifetime value.

In the Pandas Lab, we only looked at high policy claim amounts because we couldn't look into low customer lifetime values. If we had tried to work with that column, we wouldn't have been able to because customer lifetime value wasn't clean and in its proper format. So after cleaning and formatting the data, let's get some more interesting insights!

Instructions:

- Review the statistics again for total claim amount and customer lifetime value to gain an understanding of the data.
- To identify potential areas for improving customer retention and profitability, we want to focus on customers with a high policy claim amount and a low customer lifetime value. Consider customers with a high policy claim amount to be those in the top 25% of the total claim amount, and clients with a low customer lifetime value to be those in the bottom 25% of the customer lifetime value. Create a pandas DataFrame object that contains information about customers with a policy claim amount greater than the 75th percentile and a customer lifetime value in the bottom 25th percentile.
- Use DataFrame methods to calculate summary statistics about the high policy claim amount and low customer lifetime value data. To do so, select both columns of the dataframe simultaneously and pass it to the `.describe()` method. This will give you descriptive statistics, such as mean, median, standard deviation, minimum and maximum values for both columns at the same time, allowing you to compare and analyze their characteristics.

In [ ]:
# Your code here